In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [27]:
import time
import os

import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [3]:
BASE_PATH = '.'
BUFFER_SIZE = 60000
BATCH_SIZE = 256
EPOCHS = 30
NOISE_DIM = 100
SEED = tf.random.normal([16, NOISE_DIM])

In [4]:
def make_generator_model():
    model = tf.keras.Sequential()

    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(NOISE_DIM,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

generator = make_generator_model()
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1254400   
                                                                 
 batch_normalization (BatchN  (None, 12544)            50176     
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 12544)             0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 7, 7, 128)        819200    
 nspose)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 128)        5

In [5]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

discriminator = make_discriminator_model()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        1664      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [6]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [7]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [8]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [9]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [10]:
checkpoint_dir = BASE_PATH + '/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
ckptManager = tf.train.CheckpointManager(checkpoint, 'checkpoint_prefix', max_to_keep=10)

checkpoint.restore(ckptManager.latest_checkpoint)
if ckptManager.latest_checkpoint:
  print("Restored from {}".format(ckptManager.latest_checkpoint))
else:
  print("Initializing from scratch.")

Initializing from scratch.


In [11]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.savefig(f'image_at_epoch_{epoch}.png')
    plt.show()
    plt.close()

In [12]:
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [44]:
@tf.function
def train_discriminator(train_dataset):
    while True:
        output_real = np.array([])
        output_fake = np.array([])

        for image_batch in train_dataset:
            noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

            with tf.GradientTape() as disc_tape:
                generated_images = generator(noise, training=False)

                real_output = discriminator(image_batch, training=True)
                fake_output = discriminator(generated_images, training=True)
                np.concatenate((output_real,real_output))
                np.concatenate((output_fake,fake_output))

                disc_loss = discriminator_loss(real_output, fake_output)

            gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


        train_acc_metric = tf.keras.metrics.BinaryAccuracy()
        train_acc_metric.update_state(output_real, tf.ones_like(output_real))
        train_acc_metric.update_state(fake_output, tf.zeros_like(fake_output))
        if train_acc_metric.result() > 75:
            break

In [42]:
@tf.function
def train_generator(train_dataset):
    while True:
        output = np.array([])

        for image_batch in train_dataset:
            noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])

            with tf.GradientTape() as gen_tape:
                generated_images = generator(noise, training=True)

                fake_output = discriminator(generated_images, training=True)
                np.concatenate((output,fake_output))

                gen_loss = generator_loss(fake_output)

            gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
            generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

        train_acc_metric = tf.keras.metrics.BinaryAccuracy()
        train_acc_metric.update_state(output, tf.ones_like(output))
        if train_acc_metric.result() > 75:
            break

In [39]:
def train(train_dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    train_discriminator(train_dataset)
    train_generator(train_dataset)

      # if (batch + 1) % 50 == 0:
        # print (f'Finished batch {batch + 1} (size = {BATCH_SIZE}): {time.time()-start} sec')

    generate_and_save_images(generator, int(checkpoint.save_counter), SEED)

    # Save the model every 1 epoch
    if (epoch + 1) % 1 == 0:
      ckptManager.save()

    print (f'Time for epoch {epoch + 1} is {time.time()-start} sec')

In [16]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [45]:
train(train_dataset, EPOCHS)

NotImplementedError: in user code:

    File "C:\Users\piotr\AppData\Local\Temp\ipykernel_2428\3320939595.py", line 17, in train_discriminator  *
        np.concatenate((output_real,real_output))
    File "<__array_function__ internals>", line 180, in concatenate  **
        

    NotImplementedError: Cannot convert a symbolic Tensor (sequential_1/dense_1/BiasAdd:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported
